In [3]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')

## to the work directory
import os
work_dir = "/content/drive/My Drive/Earth-Engine-with-Deep-Learning/trainer/ai_platform_package"
os.chdir(work_dir)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
## Data loader
%%writefile dataLoader.py

from . import config
import tensorflow as tf

# Dataset loading functions
def parse_tfrecord(example_proto):
  return tf.io.parse_single_example(example_proto, config.Features_Dict)

def to_tuple(inputs):
  inputsList = [inputs.get(key) for key in config.Features]
  stacked = tf.stack(inputsList, axis=0)
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(config.Bands)], stacked[:,:,len(config.Bands):]

def get_dataset(pattern):
	glob = tf.io.gfile.glob(pattern)
	dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
	dataset = dataset.map(parse_tfrecord)
	dataset = dataset.map(to_tuple)
	return dataset

def get_training_dataset():
	glob = 'gs://' + config.Bucket + '/' + config.Train_Data_Folder + '/' + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(config.Buffer_Size).batch(config.Batch_Size).repeat()
	return dataset

# def get_eval_dataset():
# 	glob = 'gs://' + config.DATA_BUCKET + '/' + config.FOLDER + '/' + config.EVAL_BASE + '*'
# 	dataset = get_dataset(glob)
# 	dataset = dataset.batch(1).repeat()
# 	return dataset


Overwriting dataLoader.py
